# Beguiling West Niling

This is the Jupyter Notebook containing all the code required to complete the Kaggle West Nile Virus Challenge. All directorial references can be fulfilled by downloading the repository and maintaining folder structure.

In [138]:
import pandas as pd
import numpy as np

Importing data.

In [139]:
weather = pd.read_csv('../input/raw/weather.csv')
spray = pd.read_csv('../input/raw/spray.csv')
train = pd.read_csv('../input/raw/train.csv')
test = pd.read_csv('../input/raw/test.csv')

## Data Wrangling

### Cleaning Training Data

Converting Date to datetime format.

In [140]:
train.Date = pd.to_datetime(train.Date, infer_datetime_format=True)

Getting rid of leakage (duplicate rows when NumMosquitos is greater than 50)

In [141]:
train = train.groupby(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy', 'WnvPresent'])['NumMosquitos'].sum().reset_index()

Dropping geographical features.

In [142]:
train = train.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet'], axis=1)

Making sure that Species and Trap are formatted as Strings.

In [143]:
train.Species = train.Species.astype(str)
train.Trap = train.Trap.astype(str)

Renaming features.

In [144]:
rename_spray = {
    'Latitude': 'Lat',
    'Longitude': 'Long'
}

train = train.rename(index=str, columns=rename_spray)

Saving to a new CSV.

In [145]:
train.to_csv('../input/clean/train_cleaned.csv')

### Cleaning Testing Data

Converting Date to datetime format.

In [146]:
test.Date = pd.to_datetime(test.Date, infer_datetime_format=True)

Dropping geographical features.

In [147]:
test = test.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet'], axis=1)

Making sure that Species and Trap are formatted as Strings.

In [148]:
test.Species = test.Species.astype(str)
test.Trap = test.Trap.astype(str)

Renaming features.

In [149]:
rename_spray = {
    'Latitude': 'Lat',
    'Longitude': 'Long'
}

test = test.rename(index=str, columns=rename_spray)

Saving to a new CSV.

In [150]:
test.to_csv('../input/clean/test_cleaned.csv')

### Cleaning Weather Data

Creating a function that allows as to take all numbers and convert them from string to numerical format.

We're trying to convert everything stored as '10.0' to 10.0.

In [151]:
def make_float(x):
    
    x = x.strip()
    
    if x.isnumeric():
        return float(x)
    
    if x == 'M' or x == 'T' or x == '-':
        return None
    
    return x

Converting Date to datetime format.

In [152]:
weather.Date = pd.to_datetime(weather.Date, infer_datetime_format=True)

Converting the relevant columns to numeric formats.

In [153]:
columns_to_int = ['Station', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', \
                  'Cool', 'Sunrise', 'Sunset', 'Depth', 'SnowFall', 'PrecipTotal', 'StnPressure', \
                  'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed']

for col in columns_to_int:
    try:
        weather[col] = weather[col].astype('float')
    except:
        weather[col] = weather[col].apply(make_float)

Dropping a column.

In [154]:
weather = weather.drop(['Water1'], axis=1)

Renaming columns.

rename_weather = {
    'Tmax': 'Temp_Max',
    'Tmin': 'Temp_Min',
    'Tavg': 'Temp_Avg',
    'Depart': 'Temp_Norm_Dev',
    'DewPoint': 'Temp_Dew_Point',
    'WetBulb': 'Temp_Wet_Bulb',
    'Depth': 'Max_Snow_Depth',
}

weather = weather.rename(index=str, columns=rename_weather)

Saving to a new CSV.

In [155]:
weather.to_csv('../input/clean/weather_cleaned.csv')

### Cleaning Spray Data

Dropping Duplicates.

In [156]:
spray = spray.drop_duplicates()

Dropping time.

In [157]:
spray = spray.drop(['Time'], axis=1)

Renaming columns.

In [158]:
rename_spray = {
    'Latitude': 'Lat',
    'Longitude': 'Long'
}

spray = spray.rename(index=str, columns=rename_spray)

Saving to new CSV.

In [159]:
spray.to_csv('../input/clean/spray_cleaned.csv')

## EDA

Importing cleaned data.

In [160]:
train = pd.read_csv('../input/clean/train_cleaned.csv').drop('Unnamed: 0', axis=1).dropna(axis=1)
test = pd.read_csv('../input/clean/test_cleaned.csv').drop(['Unnamed: 0', 'Id'], axis=1).dropna(axis=1)
weather = pd.read_csv('../input/clean/weather_cleaned.csv').drop('Unnamed: 0', axis=1)
spray = pd.read_csv('../input/clean/spray_cleaned.csv').drop('Unnamed: 0', axis=1).dropna(axis=1)

Converting to datetime format.

In [161]:
train.Date = pd.to_datetime(train.Date, infer_datetime_format=True)
test.Date = pd.to_datetime(test.Date, infer_datetime_format=True)
weather.Date = pd.to_datetime(weather.Date, infer_datetime_format=True)
spray.Date = pd.to_datetime(spray.Date, infer_datetime_format=True)

## Feature Engineering

Creating a duplicated rows feature to address leakage in training data.

In [162]:
train['Duplicated_Rows'] = train.NumMosquitos.apply(lambda x : round(x / 50))

Creating the same duplicated rows feature in test data.

In [163]:
## Boolean column that simply says if a row is Duplicated
test['Duplicated'] = test.duplicated(['Date', 'Species', 'Trap', 'Lat', 'Long', 'AddressAccuracy'])

## Generating an empty column
test['Duplicated_Rows'] = test['Duplicated'].apply(lambda x : 1 if x else 0)

Function that determines how many rows are duplicated in test data for each instance.

In [164]:
def fill_duplicated(row):
    
    size = 0
    
    if row.Duplicated:
        size = test[(test.Date == row.Date) & (test.Species == row.Species) \
                    & (test.Trap == row.Trap) & (test.Lat == row.Lat) \
                    & (test.Long == row.Long)].shape[0] - 1
    
    return size

test['Duplicated_Rows'] = test.apply(fill_duplicated, axis=1)

## Dropping boolean column
test = test.drop('Duplicated', axis=1)

Creating a trap dictionary in order to factorize trap consistently over both training and test data.

In [165]:
trap_dict = {}

for i, trap in enumerate(list(test.Trap.unique())):
    trap_dict[trap]= i + 1
    
train['Trap_Fact'] = train['Trap'].apply(lambda x : trap_dict[x])
test['Trap_Fact'] = test['Trap'].apply(lambda x : trap_dict[x])

Converting Date to an ordinal feature and creating columns based on date.

In [166]:
import datetime

train['Date_Ord'] = train.Date.apply(datetime.date.toordinal)
test['Date_Ord'] = test.Date.apply(datetime.date.toordinal)

train['Week'] = train.Date.dt.week
train['Year'] = train.Date.dt.year

test['Week'] = test.Date.dt.week
test['Year'] = test.Date.dt.year

Calculating days since last checked a trap. This is a necessary feature because number of mosquitos is proportional to how long a trap hasn't been checked.

In [167]:
## Function for converting nano-seconds to days (created to deal with Numpy default datetime settings).

def nanosec2days(diff):
    return int(diff / ((10 ** 9) * 60 * 60 * 24))

For every trap, for every day, how many days since last checked. For training and test data.

In [168]:
train_trap_diffs = {}

for trap in train.Trap.unique():
#     print(trap)
    df_trap = train[train.Trap == trap]
    
    diff_dict = {}
    uniq_dates = df_trap.Date.unique()
    for i, date in enumerate(uniq_dates):
        if not i:
            diff_dict[str(date)[:10]] = 0
        else:
            diff_dict[str(date)[:10]] = nanosec2days(uniq_dates[i] - uniq_dates[i - 1])

    train_trap_diffs[trap] = diff_dict
    
train_trap_diffs

{'T001': {'2007-06-26': 0,
  '2007-07-11': 15,
  '2007-07-18': 7,
  '2007-08-01': 14,
  '2007-08-07': 6,
  '2007-08-15': 8,
  '2007-08-21': 6,
  '2007-08-24': 3,
  '2007-09-12': 19,
  '2007-10-04': 22},
 'T002': {'2007-05-29': 0,
  '2007-06-05': 7,
  '2007-06-26': 21,
  '2007-06-29': 3,
  '2007-07-02': 3,
  '2007-07-11': 9,
  '2007-07-18': 7,
  '2007-07-27': 9,
  '2007-08-01': 5,
  '2007-08-07': 6,
  '2007-08-15': 8,
  '2007-08-21': 6,
  '2007-08-24': 3,
  '2007-09-04': 11,
  '2007-09-12': 8,
  '2007-09-18': 6,
  '2007-09-24': 6,
  '2007-10-04': 10,
  '2009-05-28': 602,
  '2009-06-02': 5,
  '2009-06-05': 3,
  '2009-06-12': 7,
  '2009-06-19': 7,
  '2009-06-26': 7,
  '2009-07-06': 10,
  '2009-07-10': 4,
  '2009-07-17': 7,
  '2009-07-24': 7,
  '2009-07-31': 7,
  '2009-08-07': 7,
  '2009-08-25': 18,
  '2009-09-14': 20,
  '2009-09-17': 3,
  '2009-09-25': 8,
  '2011-06-17': 630,
  '2011-06-24': 7,
  '2011-06-30': 6,
  '2011-07-11': 11,
  '2011-07-15': 4,
  '2011-07-25': 10,
  '2011-07-29': 4

In [169]:
test_trap_diffs = {}

for trap in test.Trap.unique():
#     print(trap)
    df_trap = test[test.Trap == trap]
    
    diff_dict = {}
    uniq_dates = df_trap.Date.unique()
    for i, date in enumerate(uniq_dates):
        if not i:
            diff_dict[str(date)[:10]] = 0
        else:
            diff_dict[str(date)[:10]] = nanosec2days(uniq_dates[i] - uniq_dates[i - 1])

    test_trap_diffs[trap] = diff_dict
    
test_trap_diffs

{'T001': {'2008-06-11': 0,
  '2008-06-17': 6,
  '2008-06-24': 7,
  '2008-07-01': 7,
  '2008-07-04': 3,
  '2008-07-11': 7,
  '2008-07-14': 3,
  '2008-07-21': 7,
  '2008-07-23': 2,
  '2008-07-24': 1,
  '2008-07-28': 4,
  '2008-07-29': 1,
  '2008-08-04': 6,
  '2008-08-05': 1,
  '2008-08-12': 7,
  '2008-08-13': 1,
  '2008-08-19': 6,
  '2008-08-25': 6,
  '2008-08-26': 1,
  '2008-09-02': 7,
  '2008-09-03': 1,
  '2008-09-09': 6,
  '2008-09-15': 6,
  '2008-09-19': 4,
  '2008-09-29': 10,
  '2010-06-02': 611,
  '2010-06-11': 9,
  '2010-06-18': 7,
  '2010-06-25': 7,
  '2010-06-28': 3,
  '2010-07-01': 3,
  '2010-07-02': 1,
  '2010-07-12': 10,
  '2010-07-13': 1,
  '2010-07-16': 3,
  '2010-07-19': 3,
  '2010-07-23': 4,
  '2010-07-26': 3,
  '2010-07-29': 3,
  '2010-07-30': 1,
  '2010-08-05': 6,
  '2010-08-06': 1,
  '2010-08-13': 7,
  '2010-08-19': 6,
  '2010-08-20': 1,
  '2010-08-26': 6,
  '2010-08-27': 1,
  '2010-09-02': 6,
  '2010-09-10': 8,
  '2010-09-13': 3,
  '2010-09-16': 3,
  '2010-09-17': 1,


Functions to fill 'Days since Last Checked' for test and train data. Created multiple functions because I'm using DataFrame.apply

In [170]:
def fill_train_diffs(row):
    
    year = str(row.Date.year)
    month = str(row.Date.month)
    day = str(row.Date.day)
    
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    
    date = year + '-' + month + '-' + day
#     print(date)
    
    return train_trap_diffs[row.Trap][date]

In [171]:
def fill_test_diffs(row):
    
    year = str(row.Date.year)
    month = str(row.Date.month)
    day = str(row.Date.day)
    
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    
    date = year + '-' + month + '-' + day
#     print(date)
    
    return test_trap_diffs[row.Trap][date]

In [172]:
train['Days_Since_Last_Check'] = train.apply(fill_train_diffs, axis=1)
test['Days_Since_Last_Check'] = test.apply(fill_test_diffs, axis=1)

Dropping Species and Trap columns.

In [173]:
train = train.drop(['Species', 'Trap'], axis=1)
test = test.drop(['Species', 'Trap'], axis=1)

Created a function to calculate Spray Factor for each instance.

In [174]:
def spray_factor(row):
    
    Date = row.Date
    Lat = row.Lat
    Long = row.Long   
    
    rel = spray[(spray['Date'] < Date) & ((Date-spray['Date']) < pd.Timedelta('14 days'))]
    
    factor = 0
    for item in rel.iterrows():
        latdiff = Lat - item[1]['Lat']
        longdiff = Long - item[1]['Long']
        dis = .5 ** (latdiff ** 2 + longdiff ** 2)
        t = (Date - item[1]['Date']).days
        factor += (.5) ** (t / 1.5) / dis
    return factor

In [175]:
train['Spray_Factor'] = train.apply(spray_factor, axis=1)
test['Spray_Factor'] = test.apply(spray_factor, axis=1)

Dummifying weather codes.

In [176]:
codes = []
for code in [item.split() for item in weather.CodeSum.unique()]:
    codes += code

codes = list(set(codes))
codes

for code in codes:
    weather[code] = weather.CodeSum.apply(lambda x : 1 if code in x else 0)
   
 ## Dropping CodeSum Column
weather = weather.drop('CodeSum', axis=1)

Function that calculates length of day (will be used further down in the notebook).

In [177]:
def time_diff(row):
    
    try:
        sunset = (round(row.Sunset / 100) * 60) + (row.Sunset % 100)
        sunrise = (round(row.Sunrise / 100) * 60) + (row.Sunrise % 100)
    except:
        return None

    return int(abs(sunset - sunrise))

Function used to extrapolate average weather across stations.

In [178]:
def weather_extrap(colname):
    
    for i in range(len(weather_both[colname])):
        
          if np.isnan(weather_both[colname][i]):
                
                if not np.isnan(weather1[colname][i]):
                    weather_both[colname][i] = weather1[colname][i]
                    
                elif not np.isnan(weather2[colname][i]):
                    weather_both[colname][i] = weather2[colname][i]
                    
                else:
                    weather_both[colname][i] = 0.5 * (weather_both[colname][i - 1] + weather_both[colname][i + 1])

Creating aggregate features across both stations.

In [179]:
weather1.columns

Index(['index', 'Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart',
       'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'Depth',
       'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
       'ResultDir', 'AvgSpeed', 'GR', 'BR', 'SQ', 'FU', 'RA', 'TSRA', 'MIFG',
       'VCFG', 'BCFG', 'SN', 'VCTS', 'DZ', 'HZ', 'FG+', 'FG', 'TS'],
      dtype='object')

In [180]:
weather_both = pd.DataFrame()

weather1 = weather[weather['Station'] == 1].reset_index()
weather2 = weather[weather['Station'] == 2].reset_index()

weather_both['Date'] = weather1['Date']
weather_both[codes] = weather1[codes]
weather_both['Temp_Min_both']=.5*(weather1['Tmin'] + weather2['Tmin'])
weather_both['Temp_Max_both']=.5*(weather1['Tmax'] + weather2['Tmax'])
weather_both['Temp_Avg_both']=.5*(weather1['Tavg'] + weather2['Tavg'])
weather_both['Temp_Dew_Point_both']=.5*(weather1['DewPoint'] + weather2['DewPoint'])
weather_both['Temp_Wet_Bulb_both']=.5*(weather1['WetBulb'] + weather2['WetBulb'])
weather_both['StnPressure_both']=.5*(weather1['StnPressure'] + weather2['StnPressure'])
weather_both['Ht_Cool_both']= weather1['Heat'] - weather1['Cool']
weather_both['Day Length'] = weather1.apply(time_diff, axis=1)

Merging weather data on Date.

In [181]:
train = train.merge(weather_both, on='Date')
test = test.merge(weather_both, on='Date')

Dropping Number of Mosquitos from train data and Date from both train and test.

In [182]:
train = train.drop(['NumMosquitos', 'Date'], axis=1)
test = test.drop(['Date'], axis=1)

Creating aggregated columns with time lagged data for various functions.

This allows us to see information like 'Maximum temperature over last 14 days' or 'Average humidity over last 3 days'.

In [183]:
def create_agg_cols(colname, func, deltas):
    new_frame=pd.DataFrame()
    base_col=weather_both[colname]

    for delta in deltas:
        new_colname=colname+str(delta)
        new_frame[new_colname]=[0]*len(base_col)

        for i in range(delta,len(base_col)):
            relevant=base_col[i-delta: i]
            new_frame[new_colname][i]=func(relevant)

    return new_frame

In [ ]:
deltas=[1,2,4,7,10,14]

weather_agged = weather_both

weather_agged = pd.concat([weather_agged, create_agg_cols('Temp_Min_both', min, deltas)], axis=1)
weather_agged = pd.concat([weather_agged, create_agg_cols('Temp_Max_both', max, deltas)], axis=1)
weather_agged = pd.concat([weather_agged, create_agg_cols('Temp_Dew_Point_both', max, deltas)], axis=1)
weather_agged = pd.concat([weather_agged, create_agg_cols('Temp_Avg_both', np.mean, deltas)], axis=1)

Merging time lagged data on Date.

In [ ]:
train = train.merge(weather_agged, on='Date')
test = test.merge(weather_agged, on='Date')

## Dropping Date
train = train.drop(['Date'], axis=1)
test = test.drop(['Date'], axis=1)

Creating a count to show how often a specific Trap turns up.

In [ ]:
trap_count = train.groupby('Trap_Fact').count().Duplicated_Rows.to_dict()

for key in list(set(test.groupby('Trap_Fact').count().Duplicated_Rows.to_dict()) - set(train.groupby('Trap_Fact').count().Duplicated_Rows.to_dict().keys())):
    trap_count[key] = -1

In [ ]:
train['Trap_Count'] = train.Trap_Fact.apply(lambda x : trap_count[x])
test['Trap_Count'] = test.Trap_Fact.apply(lambda x : trap_count[x])

Creating a version of the last version trap count to show how often a specific Trap turns up in a year. These variables naturally have high correlation, but they do prove useful.

In [ ]:
trap_by_year_count = train.groupby(['Year', 'Trap_Fact']).count().Duplicated_Rows.to_dict()

for key in list(set(test.groupby('Trap_Fact').count().Duplicated_Rows.to_dict()) - set(train.groupby('Trap_Fact').count().Duplicated_Rows.to_dict().keys())):
    trap_by_year_count[(None,key)] = -1

In [ ]:
## Creating trap_by_year_count for train.

train['Trap_By_Year_Count'] = train.Year.apply(lambda x : 0)

def fill_tbyc(row):
    return trap_by_year_count[(row['Year'], row['Trap_Fact'])]
    
train['Trap_By_Year_Count'] = train.apply(fill_tbyc, axis=1)

In [ ]:
## Creating trap_by_year_count for test

In [ ]:
test['Trap_By_Year_Count'] = test.Year.apply(lambda x : 0)

def fill_tbyc(row):
    try:
        count = trap_by_year_count[(row['Year'], row['Trap_Fact'])]
    except:
        try:
            count = trap_by_year_count[(row['Year'] + 1, row['Trap_Fact'])]
        except:
            count = -1
    return count
    
test['Trap_By_Year_Count'] = test.apply(fill_tbyc, axis=1)

Replacing empty values in WetBulb and StnPressure columns with -1 to make it usable in a model.

In [ ]:
def replace_none(x):
    if x == 'NaN':
        return -1
    return x

train.Temp_Wet_Bulb_both = train.Temp_Wet_Bulb_both.apply(replace_none)
test.Temp_Wet_Bulb_both = test.Temp_Wet_Bulb_both.apply(replace_none)

train.StnPressure_both = train.StnPressure_both.apply(replace_none)
test.StnPressure_both = test.StnPressure_both.apply(replace_none)

Creating target column.

In [ ]:
target = train[['WnvPresent']]

Dropping WnV Present from train.

In [ ]:
train = train.drop(['WnvPresent'], axis=1)

Dropping Year columns - they are already represented in the ordinal dates.

In [ ]:
train = train.drop(['Year'], axis=1)
test = test.drop(['Year'], axis=1)

Saving to a final CSV.

In [ ]:
train.to_csv('../../../input/final/train_final.csv')
target.to_csv('../../../input/final/train_target_final.csv')
test.to_csv('../../../input/final/test_final.csv')

## Modeling

In [ ]:
train = pd.read_csv('../../../input/final/train_final.csv').drop('Unnamed: 0', axis=1)
target = pd.read_csv('../../../input/final/train_target_final.csv').drop('Unnamed: 0', axis=1).WnvPresent
test = pd.read_csv('../../../input/final/test_final.csv').drop('Unnamed: 0', axis=1)

Function that allows us easily write a csv by just passing in a model and a model name.

In [ ]:
def write_csv(model_name, model):
    
    model_data = [item[1] for item in model.predict_proba(test)]
    
    pd.DataFrame(data=model_data).to_csv(model_name + '.csv')
    results = pd.read_csv(model_name + '.csv')
    results.columns = ['Id', 'WnvPresent']
    results.Id = results.Id.apply(lambda x : x + 1)
    results.to_csv(model_name + '.csv', index=False)

### XGBoost

In [ ]:
from xgboost import plot_importance
from xgboost.sklearn import XGBClassifier
from matplotlib import pyplot as plt

In [ ]:
xgb1 = XGBClassifier(learning_rate=0.2, seed=42, silent=1) # 0.2 best so far

In [ ]:
xgb1.fit(train, target)

# plot feature importance
fig, ax = plt.subplots(figsize=(10, 15))
plot_importance(xgb1, ax=ax)
plt.show()

Removing all columns with 0 feature importance. This should make no difference to the XGBoost model (apart from making things easier to read and more manageable).

In [ ]:
importance = dict(zip(train.columns, xgb1.feature_importances_ * 100 / max(xgb1.feature_importances_)))

for key in importance.keys():
    if importance[key] == 0:
        train = train.drop([key], axis=1)
        test = test.drop([key], axis=1)

In [ ]:
xgb1.fit(train, target)

# plot feature importance
fig, ax = plt.subplots(figsize=(10, 15))
plot_importance(xgb1, ax=ax)
plt.show()

In [ ]:
write_csv('xgboost', xgb1)